In [15]:
import numpy as np
import pandas as pd

In [16]:
intervalsDelimiter = ";"
# intervals = [["100;105", 3],["105;110",4],["110;115",9],["115;120",6],["120;125",2],["125;130",1]]
#intervals = [["60;63",5],["63;66",18],["66;69",42],["69;72",27],["72;75",8]]
#dfIntervals = {
#    "intervals" : ["60;63","63;66","66;69","69;72","72;75"],
#    "frecuencies" : [5,18,42,27,8]
#}

dfIntervals = {
    "intervals": ["10;19","19;28","28;37","37;46","46;55","55;64","64;73"],
    "frecuencies":[5,11,8,5,8,6,7]
}

In [29]:
class StatisticalTable():
    dataFrame = None
    totalItems = 0
    accumulatedFrecuency = 0
    
    def __init__(self, intervals):
        self.dataFrame = pd.DataFrame(intervals)
    
    def createTable(self):
        total = 0
        accumulatedFrecuency = [total := total + x for x in self.dataFrame['frecuencies']]
        self.accumulatedFrecuency = total
        xi = self.calculateMarkClass(self.dataFrame['intervals'])
        self.dataFrame['Fi'] = accumulatedFrecuency
        frec = np.array(self.dataFrame['frecuencies'])
        xifi = xi * frec
        self.dataFrame['xi'] = xi
        self.dataFrame['xi * fi'] = xifi
        return self.dataFrame
    
    def getTotalItems(self):
        self.totalItems = len(self.dataFrame['intervals'])
        return len(self.dataFrame['intervals'])
    
    def calculateMarkClass(self,intervals):
        classes = np.zeros((1,len(intervals)))
        for i in range(len(intervals)):
            classMark = 0
            interval = intervals[i].split(intervalsDelimiter)
            minNumber = float(interval[0])
            maxNumber = float(interval[1])
            classMark = minNumber + (maxNumber-minNumber)/2
            classes[0][i] = classMark
        return classes[0]
    
    def calculateMean(self):
        total = 0
        [total := total + x for x in self.dataFrame['xi * fi']]
        return total/self.accumulatedFrecuency
    
    # This function return the current index of the selected interval
    def getModeInterval(self):
        n = self.accumulatedFrecuency / 2 # In the calc of the median, we need to positionate in the half of the interval
        currentAcumulatedFrec = None
        prevAccumulatedFrec = None
        nextAccumulatedFrec = None
        frecuencies =  self.dataFrame['frecuencies']
        max_value = max(frecuencies)

        for i in range(len(frecuencies)):
            if (frecuencies[i] == max_value):
                print(i)
                return i
        # es el que corresponde al a frecuencia más alta
    def getMedianInterval(self):
        n = self.accumulatedFrecuency / 2 # In the calc of the median, we need to positionate in the half of the interval
        currentAcumulatedFrec = None
        prevAccumulatedFrec = None
        nextAccumulatedFrec = None
        frecuencies =  self.dataFrame['frecuencies']
        accumulatedFrecuencies = self.dataFrame['Fi']
        for i in range(len(frecuencies)):
            currentAccumulatedFrec = accumulatedFrecuencies[i]
            if(i < len(frecuencies)):
                nextAccumulatedFrec = accumulatedFrecuencies[i+1]
            else:
                nextAccumulatedFrec = accumulatedFrecuencies[-1] # last acc frecuency

            if(i > 0):
                prevAccumulatedFrec = accumulatedFrecuencies[i-1]
            else:
                prevAccumulatedFrec = frecuencies[i] # i == 0, the acc frecuency is the normal frecuency
            if(n == currentAccumulatedFrec):
                # if n is exact currentAccumulated i choose that interval
                return i
            
            if(n > currentAccumulatedFrec and n < nextAccumulatedFrec):
                # if the half accumulated frecuency is greater than current and less than next, i+1 is the candidate
                return i+1

    def calculateMedian(self):
        median = None
        selectedIntervalIndex = self.getMedianInterval() 
        selectedInterval = self.dataFrame['intervals'][selectedIntervalIndex].split(intervalsDelimiter)
        li = float(selectedInterval[0]) # inferior limit of the current interval
        ls = float(selectedInterval[1])
        fi = float(self.dataFrame['frecuencies'][selectedIntervalIndex]) # frecuency of the current interval
        accumulatedPrevFrec = self.dataFrame['Fi'][selectedIntervalIndex-1]
        amplitud = ls -li # Amplitud expressed as intervals difference
        median = li + ((((self.accumulatedFrecuency/2) - accumulatedPrevFrec))/fi)*amplitud
        return median            

    def calculateMode(self):
        mode = None
        selectedIntervalIndex = self.getModeInterval() 
        
        currentFi = self.dataFrame['frecuencies'][selectedIntervalIndex]
        nextFi = self.dataFrame['frecuencies'][selectedIntervalIndex+1]
        prevFi = self.dataFrame['frecuencies'][selectedIntervalIndex-1]
        selectedInterval = self.dataFrame['intervals'][selectedIntervalIndex].split(intervalsDelimiter)
        
        li = float(selectedInterval[0])
        ls = float(selectedInterval[1])
        amplitud = ls - li
        di = currentFi - prevFi
        nextdi = currentFi - nextFi
        mode = li + ((di)/(di+nextdi))*amplitud
        return mode

statisticalTable = StatisticalTable(dfIntervals)
dataf = statisticalTable.createTable()
mean = statisticalTable.calculateMean()
median = statisticalTable.calculateMedian()
mode = statisticalTable.calculateMode()

print(dataf)
print("Media: {}, Mediana: {}, Moda {}".format(mean,median,mode))

1
  intervals  frecuencies  Fi    xi  xi * fi
0     10;19            5   5  14.5     72.5
1     19;28           11  16  23.5    258.5
2     28;37            8  24  32.5    260.0
3     37;46            5  29  41.5    207.5
4     46;55            8  37  50.5    404.0
5     55;64            6  43  59.5    357.0
6     64;73            7  50  68.5    479.5
Media: 40.78, Mediana: 38.8, Moda 25.0
